In [1]:
import plotly.express as px
import numpy as np
import pandas as pd
import DataClean
import datetime
import os
# import plotly.graph_objects as go

In [24]:
import pickle

In [25]:
dat = pickle.load(open('compiled_data.p', 'rb'))
countries = list(dat.index.get_level_values(0).unique())
stats = list(dat.columns)
stat_map = {
    'DailyPosTestRate': 'Daily Positive Test Rate',
    'Confirmed': 'Confirmed Tests',
    'NewTests': 'Daily New Tests',
    'CumulativePosTestRate': 'Cumulative Positive Test Rate',
    'NewCases': 'New Cases',
    'TotalTests': 'Total Tests',
    'ConfirmedGrowth': 'Daily Confirmed Cases Growth (%)',
    'CumulativeDeathRate': 'Cumulative Death Rate',
    'HospitalizationRate': 'Hospitalization Rate',
    'TotalTestsPer10k': 'Total Tests per 10k Population',
    'ConfirmedPer10k': 'Total Confirmed per 10k Population',
    'NewTestsPer10k': 'Daily Tests per 10k Population'
}

xs = ['DaysSinceFirst', 'Date', 'DaysSinceTenthDeath', 'DaysSinceShutdown']
xs_map = {'DaysSinceFirst': 'Days Since First Confirmed',
         'DaysSinceTenthDeath': 'Days Since 10th Death',
         'DaysSinceShutdown': 'Days Since Economic Shutdown'}

# Important Dates by Country:

### Peru
- Sourhces: ministerio de salud, wikipedia page
* March 11 - Declaran emergencia sanitaria por 90 dias
* March 13 - Suspenden vuelos internacional desde Europa y Asia
* March 16 - Teletrabajo para todos menos los esenciales
* March 17 - Cierran fronteras
* March 18 - Cuarentena general entre 8PM y 5 AM.
* https://www.dge.gob.pe/portal/

### Panama
* Something

### Guatemala
- Source: Alejandro Giammattei twitter
* March 22 - Cuarentena general entre 4 PM y 4 AM por 8 dias

### Testing Analysis stuff

In [168]:
import requests
from sodapy import Socrata
import pandas as pd
import numpy as np
import datetime

In [142]:
client = Socrata("datos.gov.co",
                 "aSEItRti2CveuMdUcoBfyeWTQ",
                )
#                  user = "abishalom",
#                  password = "PAi#%4NHPQhu4z")

results = client.get("gt2j-8ykr", limit = 100000)

results = pd.DataFrame.from_records(results)

# results

In [143]:
#filter out babies that have ages in months
results = results[~results.edad.str.contains('mes')]

new_map = {
    'fecha_de_diagn_stico': 'fecha_de_diagnostico',
    'ciudad_de_ubicaci_n' : 'ciudad_de_ubicacion',
    'atenci_n': 'atencion',
    'pa_s_de_procedencia': ' pais_de_procedencia'
}
results = results.rename(columns = new_map)
results.fecha_de_diagnostico = pd.to_datetime(results.fecha_de_diagnostico)
results.fecha_recuperado = results.fecha_recuperado.replace({'No': np.nan})
results.fecha_recuperado = pd.to_datetime(results.fecha_recuperado)
results.fis = results.fis.replace(['Asintomático', 'No disponible'], np.nan)
results.fis = pd.to_datetime(results.fis)
results.fecha_de_muerte = pd.to_datetime(results.fecha_de_muerte)

results.id_de_caso = results.id_de_caso.astype('int32')
results.edad = results.edad.astype('int32')

results = results.set_index("id_de_caso")

In [156]:
people = results.drop('atencion', axis=1)
people
current_status = results[['atencion']].rename(columns = {'atencion': datetime.date.today()})
current_status

,2020-04-19
id_de_caso,
1,Recuperado
2,Recuperado
3,Recuperado
4,Recuperado
5,Recuperado
...,...
3617,Casa
3618,Hospital
3619,Casa


In [158]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite://', echo=False)

Base = declarative_base()

In [159]:
people.to_sql('people', con = engine,
               if_exists = 'replace',
               index = True)
current_status.to_sql('current_status', con = engine,
                     if_exists = 'replace',
                     index = True)
# engine.execute('SELECT * FROM people').fetchall()

# engine

In [161]:
current_status = results[['atencion']].rename(columns = {'atencion': datetime.date.today() - datetime.timedelta(days = 1)})
current_status

,2020-04-18
id_de_caso,
1,Recuperado
2,Recuperado
3,Recuperado
4,Recuperado
5,Recuperado
...,...
3617,Casa
3618,Hospital
3619,Casa


In [162]:
current_status.to_sql('current_status', con = engine,
                     if_exists = 'replace',
                     index = True)

In [166]:
engine.execute('SELECT * FROM current_status').fetchall()

[(1, 'Recuperado'),
 (2, 'Recuperado'),
 (3, 'Recuperado'),
 (4, 'Recuperado'),
 (5, 'Recuperado'),
 (6, 'Recuperado'),
 (7, 'Recuperado'),
 (8, 'Recuperado'),
 (9, 'Recuperado'),
 (10, 'Recuperado'),
 (11, 'Recuperado'),
 (12, 'Recuperado'),
 (13, 'Recuperado'),
 (14, 'Recuperado'),
 (15, 'Recuperado'),
 (16, 'Recuperado'),
 (17, 'Recuperado'),
 (18, 'Recuperado'),
 (19, 'Recuperado'),
 (20, 'Recuperado'),
 (21, 'Recuperado'),
 (22, 'Recuperado'),
 (23, 'Recuperado'),
 (24, 'Recuperado'),
 (25, 'Casa'),
 (26, 'Recuperado'),
 (27, 'Recuperado'),
 (28, 'Recuperado'),
 (29, 'Recuperado'),
 (30, 'Recuperado'),
 (31, 'Recuperado'),
 (32, 'Recuperado'),
 (33, 'Casa'),
 (34, 'Recuperado'),
 (35, 'Recuperado'),
 (36, 'Recuperado'),
 (37, 'Recuperado'),
 (38, 'Recuperado'),
 (39, 'Recuperado'),
 (40, 'Recuperado'),
 (41, 'Recuperado'),
 (42, 'Recuperado'),
 (43, 'Recuperado'),
 (44, 'Recuperado'),
 (45, 'Recuperado'),
 (46, 'Casa'),
 (47, 'Recuperado'),
 (48, 'Recuperado'),
 (49, 'Recuperado')